In [1]:
# Dependencies
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import param

# EPA CO search base URL
base_url = "https://aqs.epa.gov/data/api/dailyData/byCounty?email=rob_mm@msn.com&key=khakiosprey53"



In [3]:
def pull_data(param, param_code):
    url = base_url + f"&param={param_code}&bdate=20240101&edate=20240229&state=37&county=183"
    response = requests.get(url).json()

    state = []; county = []
    latitude = []; longitude = []
    date = []; parameter = []
    arithmetic_mean = []
    units_of_measure = []
    sample_duration_code = []

    for row in range(len(response["Data"])):
        date.append(response["Data"][row]["date_local"])
        state.append(response["Data"][row]["state"])
        county.append(response["Data"][row]["county"])
        latitude.append(response["Data"][row]["latitude"])
        longitude.append(response["Data"][row]["longitude"])
        parameter.append(response["Data"][row]["parameter"])
        arithmetic_mean.append(response["Data"][row]["arithmetic_mean"])
        units_of_measure.append(response["Data"][row]["units_of_measure"])
        sample_duration_code.append(response["Data"][row]["sample_duration_code"])

    # create DataFrame
    df = pd.DataFrame({
        "date": date,
        "state": state,
        "county": county,
        "latitude": latitude,
        "longitude": longitude,
        "sample_duration_code": sample_duration_code,
        "parameter": parameter,
        "arithmetic_mean": arithmetic_mean,
        "units_of_measure": units_of_measure
    })

    return df

In [ ]:
# CO_df = pull_data('CO', param_code=42101)
# SO2_df = pull_data('SO2', param_code=42401)
# NO2_df = pull_data('NO2', param_code=42602)
# PM10_df = pull_data('PM10', param_code=81102)
# PM25_df = pull_data('PM2.5', param_code=88101)
PM25_df

In [12]:
# drop duplicate rows
CO_df = CO_df.drop_duplicates()
SO2_df = SO2_df.drop_duplicates()
NO2_df = NO2_df.drop_duplicates()
PM10_df = PM10_df.drop_duplicates()
PM25_df = PM25_df.drop_duplicates()

In [ ]:
# Merge DataFrames
pollution_df = CO_df.merge(SO2_df, how='inner', on=['date', 'state', 'county', 'latitude', 'longitude', 'sample_duration_code'], suffixes=('_CO', '_SO2'))
PM_df = PM10_df.merge(PM25_df, how='inner', on=['date', 'state', 'county', 'latitude', 'longitude', 'sample_duration_code'], suffixes=('_PM10', '_PM25'))
pollution_df = pollution_df.merge(PM_df, how='inner', on=['date', 'state', 'county', 'latitude', 'longitude', 'sample_duration_code'], suffixes=('_CO', '_SO2'))
pollution_df = pollution_df.merge(NO2_df, how='inner', on=['date', 'state', 'county', 'latitude', 'longitude', 'sample_duration_code'])

In [20]:
pollution_df.loc[(pollution_df['date'] == '2024-01-01')]
# pollution_df.loc[(pollution_df['date'] == '2024-01-01') | (pollution_df['date'] == '2024-02-01')]


,date,state,county,latitude,longitude,sample_duration_code,parameter_CO,arithmetic_mean_CO,units_of_measure_CO,parameter_SO2,...,units_of_measure_SO2,parameter,arithmetic_mean,units_of_measure,parameter_PM10,arithmetic_mean_PM10,units_of_measure_PM10,parameter_PM25,arithmetic_mean_PM25,units_of_measure_PM25
0,2024-01-01,North Carolina,Wake,35.856111,-78.574167,1,Carbon monoxide,0.293391,Parts per million,Sulfur dioxide,...,Parts per billion,Nitrogen dioxide (NO2),7.630435,Parts per billion,PM10 Total 0-10um STP,18.0625,Micrograms/cubic meter (25 C),PM2.5 - Local Conditions,14.4625,Micrograms/cubic meter (LC)


In [23]:
pollution_df.info()

# pollution_df.value_counts(subset=['date'])
# pollution_df['date'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   37 non-null     object 
 1   state                  37 non-null     object 
 2   county                 37 non-null     object 
 3   latitude               37 non-null     float64
 4   longitude              37 non-null     float64
 5   sample_duration_code   37 non-null     object 
 6   parameter_CO           37 non-null     object 
 7   arithmetic_mean_CO     37 non-null     float64
 8   units_of_measure_CO    37 non-null     object 
 9   parameter_SO2          37 non-null     object 
 10  arithmetic_mean_SO2    37 non-null     float64
 11  units_of_measure_SO2   37 non-null     object 
 12  parameter_PM10         37 non-null     object 
 13  arithmetic_mean_PM10   37 non-null     float64
 14  units_of_measure_PM10  37 non-null     object 
 15  paramete